# Simplification of sets of strategies

Auteur : Philippe Mathieu, [CRISTAL Lab](http://www.cristal.univ-lille.fr), [SMAC team](https://www.cristal.univ-lille.fr/?rubrique27&eid=17), [Lille University](http://www.univ-lille.fr), email : philippe.mathieu@univ-lille.fr

Contributors : Jean-Paul Delahaye (CRISTAL/SMAC), Céline Petitpré (CRISTAL/SMAC)

Creation : 2019-11-10

In [ ]:
%run ../src/game.py
%run ../src/ipd.py
%run ../src/strategies.py
%run ../src/tools.py
g.prettyPrint()   # prisoner's dilemma

Two strategies in the iterated prisoner's dilemma are equivalent if, regardless of the opponent and regardless of the length of the game, they play exactly the same rounds. The sets of strategies we are dealing with are therefore sometimes redundant: they contain the same strategies written in different ways. It therefore seems interesting to be able to simplify sets of strategies by removing the duplicates they may contain. 
Unfortunately, as everyone knows since Turing and Rice, **the equivalence of two programs is undecidable**. So there is no perfect test. Nevertheless, it is possible to provide tools to move forward in this simplification problem. This is what we will see here.

## Approximate tests of strategy equivalence

To find out if two strategies are different, just play them against a reference strategy and make sure they play differently against that opponent. If they play differently, they are proven to be different. But if they play in the same way, nothing can be concluded from it because it is perhaps that the reference strategy has not been able to reveal the difference in their behaviour. The equivalence of two strategies is therefore semi-decidable. 

The `testEquivUnit` function performs this test. It is passed a pair of strategies to test and a reference `opponent` strategy, and it compares the two strategies during `length` rounds of a meeting. It returns a boolean: equivalent (with doubt) or not.

In [ ]:
def testEquivUnit(strategies, opponent, length):
    sA,sB = strategies
    rounds1 = []
    rounds2  = []
    m1 = Meeting(g, sA, opponent, length)
    m1.run()
    m2 = Meeting(g, sB, opponent, length)
    m2.run()
    if m1.s1_score == m2.s1_score :
        if m1.s1_rounds == m2.s1_rounds :
            return True
    return False


# example of use
print(testEquivUnit((Tft(), Spiteful())  , Periodic("CCDCD"), 100))     # False
print(testEquivUnit((Tft(), Mem(0,1,"cCD")),  Periodic("CCDCD"), 100))  # true

The choice of the reference strategy is crucial. If it is too "weak", it does not allow the two compared strategies to "reveal" themselves, and then indicates that they are equivalent when they are not, as below: `Tft` and `Spiteful` are clearly different, and yet, when faced with `All_C` they have the same behavior.


In [ ]:
testEquivUnit((Tft(), Spiteful()), Periodic('C'), 100)   
# says True, but it's False

A more robust test could be to use both strategies against Periodic('CCD'). The latter allows `Tft` and `Spiteful` to reveal their true behavior. This time we have the proof that these two strategies are not equivalent.

In [ ]:
testEquivUnit((Tft(), Spiteful()), Periodic('CD'), 100)

### Exercise 1

Two strategies can of course obtain the same score against a common opponent, while having played different moves. Can you identify such a case? We can use the `itertools' package and its `permutations' method which allows to easily take 3 strategies among `n' : two for the strategies to be identified and one for the common opponent.

In [ ]:
def test2(strategies, opponent, length):
    sA,sB = strategies
    rounds1 = []
    rounds2  = []
    m1 = Meeting(g, sA, opponent, length)
    m1.run()
    m2 = Meeting(g, sB, opponent, length)
    m2.run()
    if m1.s1_score == m2.s1_score :
        if m1.s1_rounds != m2.s1_rounds :  # THE TEST HAS CHANGED COMPARED TO testEquivUnit
            return True
    return False


# example of use
bag = getMem(1,1)
bags = itertools.permutations(bag, 3)
for b in bags : 
    if test2((b[0], b[1]), b[2], 10):
        print(b[0].name + " and "+b[1].name+" produce the same score but don't play the same rounds against : "+b[2].name)            
        break

        
        
#sA = Mem(1,1,"CCCCC")
#sB = Mem(1,1,"CCDDC")
#opponent = Mem(1,1,"DCDCC")
#m = Meeting(g,sA,opponent,10)
#m.run()
#m.prettyPrint()
#m = Meeting(g,sB,opponent,10)
#m.run()
#m.prettyPrint()

In order to improve this comparison of two strategies, it is better to compare them not to a single reference strategy, but to a set of reference strategies. We compare our two strategies against each element of this set, which is surely a more robust test. Of course, as soon as one of these strategies indicates a difference between two 2 strategies, the test can stop.
The function `testEquivMultiple` works as before, but this time by trying to find a difference in behaviour thanks to a list of opponents. As before, it returns a boolean: equivalent (with doubt) or not.

In [ ]:
def testEquivMultiple(strategies, opponents, length):
    for opponent in opponents : 
        equiv = testEquivUnit(strategies, opponent, length)
        if equiv == False :
            return False
    return True

# example of use
# Same test than previously, but this time, it says False
testEquivMultiple((Tft(), Spiteful()),[Periodic('C'), Periodic('CDCCDDC'), Periodic('DDCDCDD')], 10)

To simplify a set of strategies, it is now sufficient to perform the previous test on all possible pairs. The strategies identified as potentially equivalent are then grouped together.
The `classesEquiv(l, opponents, length)` function performs this test on the `l` set. It returns the identified equivalence classes in a dictionary.
For example, if we have strat1 equivalent to strat2 and strat3 which has no equivalent, the function will return a dictionary: `{strat1 : [strat2] , strat3 : []}`

The set of keys in this dictionary constitutes the simplified set of strategies, and each entry in the dictionary corresponds to a set of equivalent strategies. For complete sets of `Mem(X,Y)` strategies, for reasons of symmetry, the number of strategies equivalent to a fixed strategy is always a power of 2 (0, 2, 4, 8, 16 ...).

In [ ]:
def classesEquiv(l, opponents, length):
    m = dict()
    while len(l) > 0 :
        m[l[0]] = []
        ind = [0]
        for j in range(len(l[1:])):
            if testEquivMultiple([l[0], l[j + 1]], opponents, length):
                m[l[0]] += [l[j + 1]]
                ind += [j + 1]
        ltmp = []
        for i in range(len(l)):
            if i not in ind :
                ltmp += [l[i]]
        l = ltmp
    return m


# This function allows you to display the names of the strategies instead of the instance number.
def printDict(ce):
    for key in ce.keys() :
        if len(ce[key]) > 0:
            print("\n" + key.name + " : " , end =" " )
        else :
            print("\n"+ key.name + ": []"  , end =" ")
        for value in ce[key]:
            print(value.name , end =" ")
    print(" ")
    

# example of use
# There is one Mem equivalent to Tft and one Mem equivalent to Spiteful. This lead 3 classes
L = [Tft(), Spiteful(), Mem(0,1,"cCD"),  Mem(1,1,"cCDDD"), Periodic("CDC") ]
ce = classesEquiv(L, [Periodic('CDCCDDC'), Periodic('DDCDCDD')], 10)
printDict(ce)
print("Simplified set size : " + str(len(ce.keys())))

This method of simplification works with any strategies (Memory type or others). This is a big advantage.

It is recalled that the quality of this simplification depends very much on the quality of the reference list. This can be easily seen when trying to simplify `mem(1,2)` in face of an ever-growing list of reference strategies.

In [ ]:
# Mem(1,2) contains 1024 strategies

# Without any opponent, they are all considered equivalent.
ce = classesEquiv(getMem(1,2), [], 10)
print(len(ce.keys()))

# Comparing with simply ALL_C , only 9 different strategies are available
ce = classesEquiv(getMem(1,2), [Periodic('C')], 10)
print(len(ce.keys()))

# We're gradually strengthening the test
ce = classesEquiv(getMem(1,2), [Periodic('C'), Periodic('CDCCDDC'), Periodic('DDCDCDD')], 10)
print(len(ce.keys()))

ce = classesEquiv(getMem(1,2), [Periodic('C'), Periodic('CDCCDDC'), Periodic('DDCDCDD'),Gradual()], 10)
print(len(ce.keys()))

# So? How large is this simplified set really?


The test can be further strengthened by starting by playing all the strategies in the test set against each other. Of course, if strategies are identical, they must have the same score in this tournament. First, we make a tournament of all the strategies together, then we test equivalencies on sets of strategies that have the same score. There is no need to test the others. Adding this equivalence of scores strengthens our test even more.
Obviously this is done at the expense of the calculation time.

The `simplifyWithTournament` function does this job. It works as before, but starts with a tournament to identify strategy packages with the same score. We then concatenate each of the dictionaries.

In [ ]:
def simplifyWithTournament(l, opponents, length):
    scores = dict()
    t = Tournament(g, opponents + l, length)
    t.run()
    res = t.matrix['Total']
    for strat in l : 
        score = res[strat.name]
        if score not in scores :
            scores[score] = [strat]
        else : 
            scores[score] += [strat]
    
    d = dict()
    for item in scores.values():
        # if more than one strategy have the same score, test classesEquiv
        if len(item) > 1 :
            res = classesEquiv(item, opponents, length)
            for it in res.keys():
                d[it] = res[it]
        else : 
            d[item[0]] = []
    return d

        
    
# example of use        
#strats = simplifyWithTournament(getMem(1,2) , [Periodic('CDCCDDC'), Periodic('DDCDCDD')], 10)
#print("Simplified set size : " + str(len(strats)))

# RECORD BROKEN! 820... but this test remains undecidable, it is nevertheless subject to a doubt...       
# printDict(strats)  

### Synthesis table

Given a basis for comparison, it is easy to display a table containing for each of the classic Memory classes, the synthesis of the sizes obtained after simulation via `ClassesEquiv` and via `SimplifyWithTournament` in order to see the difference.

In [ ]:
base  = [Periodic('CDCCDDC'), Periodic('DDCDCDD'),Gradual()]

Mem01 = getMem(0,1)
Mem10 = getMem(1,0)
Mem11 = getMem(1,1)
Mem12 = getMem(1,2)
Mem21 = getMem(2,1)

ce01 = classesEquiv(Mem01, base, 10)
ce10 = classesEquiv(Mem10, base, 10)
ce11 = classesEquiv(Mem11, base, 10)
ce12 = classesEquiv(Mem12, base, 10)
ce21 = classesEquiv(Mem21, base, 10)

simp01 = simplifyWithTournament(Mem01, base, 10)
simp10 = simplifyWithTournament(Mem10, base, 10)
simp11 = simplifyWithTournament(Mem11, base, 10)
simp12 = simplifyWithTournament(Mem12, base, 10)
simp21 = simplifyWithTournament(Mem21, base, 10)

# idem avec simplify

tab = pd.DataFrame(
        np.nan, ["Mem 0 1","Mem 1 0","Mem 1 1", "Mem 1 2", "Mem 2 1"], ["All strategies", "After classesEquiv","After simplifyWithTournament"]
    )
tab.at["Mem 0 1", "All strategies" ] = len(Mem01)
tab.at["Mem 1 0", "All strategies" ] = len(Mem10)
tab.at["Mem 1 1", "All strategies" ] = len(Mem11)
tab.at["Mem 1 2", "All strategies" ] = len(Mem12)
tab.at["Mem 2 1", "All strategies" ] = len(Mem21)
tab.at["Mem 0 1", "After classesEquiv" ] = len(ce01.keys())
tab.at["Mem 1 0", "After classesEquiv" ] = len(ce10.keys())
tab.at["Mem 1 1", "After classesEquiv" ] = len(ce11.keys())
tab.at["Mem 1 2", "After classesEquiv" ] = len(ce12.keys())
tab.at["Mem 2 1", "After classesEquiv" ] = len(ce21.keys())
tab.at["Mem 0 1", "After simplifyWithTournament" ] = len(simp01.keys())
tab.at["Mem 1 0", "After simplifyWithTournament" ] = len(simp10.keys())
tab.at["Mem 1 1", "After simplifyWithTournament" ] = len(simp11.keys())
tab.at["Mem 1 2", "After simplifyWithTournament" ] = len(simp12.keys())
tab.at["Mem 2 1", "After simplifyWithTournament" ] = len(simp21.keys())
tab

### Improve the display of strategies

Equivalence classes make it possible to highlight sets of strategies that play identically. In the case of `Mem(X,Y)`, by construction, these identical strategies have part of their genotype in common.

It is then interesting to display the sets of equivalent strategies by one and the same representation by replacing the moves which are not impacting by * . We pass to this method a set of strategies identified as equivalent, the method merges these names to provide a notation with * . Warning: this method only works for `Mem(X,Y)`, does not calculate and does not check for equivalence, it assumes that the strategies provided are equivalent.

In [ ]:
def computeStar(strategies):
    l = math.log2(len(strategies))
    assert l == int(l)
    ind_stars = []
    len_genome = len(strategies[0].genome)
    amorce = max(strategies[0].x, strategies[0].y)
    for i in range(amorce, len_genome):
        letter = strategies[0].genome[i]
        for j in range(1, len(strategies)):
            if letter != strategies[j].genome[i]:
                ind_stars += [i]
                break
    #assert len(ind_stars) == l
    new_genome = strategies[0].genome[0 : amorce ]
    for i in range(amorce, len_genome):
        if i not in ind_stars:
            new_genome += strategies[0].genome[i]
        else :
            new_genome += "*"
    #print(new_genome)
    return new_genome


# example of use
print(computeStar([Mem(1,1,"cCDCD"),Mem(1,1,"cCDCC")]))
print(computeStar([Mem(1,2,"CDCCDDDCDC"),Mem(1,2,"CDCDDDDCDC"),Mem(1,2,"CDDCDDDCDC"),Mem(1,2,"CDDDDDDCDC")]))
print(computeStar([Mem(1,2,"DDDDCCCDCD"),Mem(1,2,"DDDDCDCDCD"),Mem(1,2,"DDDDDCCDCD"),Mem(1,2,"DDDDDDCDCD")]))

print(computeStar([Mem(1,2,"ccCDCDCCDD"),Mem(1,2,"ccDDDDCDDC")])) 

## Evolution of a complete class, evolution of this class once simplified

We now have all the tools to not only simplify a class, but also to replay this simplified class, and compare the results.
Let's take the example of `Mem(1,1)`. This class contains 32 strategies. Once simplified, there are 26 strategies left. Thanks to `computeStar` one can now put the generic name in the legend. We can see that the two ecological competitions give the same 4 first ones. We notice that in the simplified class the All_C survive, while they disappear in the full class.

In [ ]:
# Simplified class computation
mem11 = getMem(1,1)
simpl = simplifyWithTournament(mem11, [Periodic('CDCCDDC'), Periodic('DDCDCDD')], 10)
print(len(simpl))

# Replacement of names by generic names
bag = []
for key in simpl.keys():
    if len(simpl[key]) > 0:
        name = computeStar([key]+simpl[key])
        bag += [key.clone(name)]
    else :
        bag += [key]

# Evolution of the initial class
e1 = Ecological(g, mem11)
e1.run()

evol=e1.historic
nbSurvivors = len(evol.iloc[-1][evol.iloc[-1]>0])
e1.drawPlot(None,nbSurvivors)

# Evolution of the simplified class with its generic names
e2 = Ecological(g, bag)
e2.run()

evol=e2.historic
nbSurvivors = len(evol.iloc[-1][evol.iloc[-1]>0])
e2.drawPlot(None,10)


## A gradual simplification

In the case of the big classes it is impossible to make the tournament work and therefore the method of simplification by the tournament cannot work. Nevertheless, one can take only a reasonable part of the class one is interested in, simplify it by tournament. We add a certain number of strategies to this simplified set without exceeding the reasonable maximum size we have set, and we start again. If at a given moment no simplification is possible, this method fails in simplification. It may be necessary to change the order in which the strategies are taken (in the hope that this order will encourage simplification), or if possible, to increase the maximum size that one has set for oneself. Obviously, this method only works if the size you set is larger than the size of the simplified set you want to obtain (820 minimum for `Mem(1,2)` for example).

In [ ]:
def simplifyStepByStep(l, opponents, length, maxSize):
    size = len(l)
    strats = []
    simplified = dict()
    end = False
    current = 0
    while not end:
        if current + maxSize > size :
            end = True
            strats += l[current :]
        else : 
            strats += l[current  : current +  maxSize]
            current = current + maxSize
        res = simplifyWithTournament(strats, opponents, length)
        print("{} strategies deleted".format(len(strats) - len(res)))
        for strat in res.keys() :
            if strat not in simplified : 
                simplified[strat] = res[strat]
            else : 
                simplified[strat] += res[strat]
        if (len(simplified) > maxSize):
            print("Impossible to continue")
            return simplified
        strats = list(simplified.keys())
    
    return simplified


        
printDict(simplifyStepByStep([Tft(), Spiteful(), Mem(0,1,"cCD"),  Mem(1,1,"cCDDD"), Periodic("CDC"), Periodic('C') ], [Periodic('CCD'), Periodic('DDC')] , 10, 4))

For relatively simple classes, it can of course be verified that overall simplification is equivalent to step-by-step simplification.

In [ ]:
res_sbs = simplifyStepByStep(getMem(1,2)[0:300], [Periodic('CCD'), Periodic('DDC')] , 10, 250) # At least 232
res = simplifyWithTournament(getMem(1,2)[0:300], [Periodic('CCD'), Periodic('DDC')], 10)
print(len(res_sbs) == len(res))
print(len(res_sbs))
print(len(res))


In [ ]:
res_sbs = simplifyStepByStep(getMem(1,1), [Periodic('CCD'), Periodic('DDC')] , 10, 30) # at least 26
res = simplifyWithTournament(getMem(1,1), [Periodic('CCD'), Periodic('DDC')], 10)
print(len(res_sbs) == len(res))

Of course, this gradual simplification depends on the order in which the strategies are brought in. One order may therefore be more conducive to simplification than another. It is therefore interesting to test several of them.

In [ ]:
def sort1_Mem(strategies):
    strategies.sort(key=lambda x: x.genome)
    return strategies

def sort2_Mem(strategies):
    strategies.sort(key=lambda x: x.genome, reverse=True)
    return strategies

def sort3_Mem(strategies):
    strategies.sort(key=lambda x: ''.join(reversed(x.genome)))
    return strategies

def sort4_Mem(strategies):
    strategies.sort(key=lambda x: ''.join(reversed(x.genome)), reverse=True)
    return strategies


sortMem11 = sort4_Mem(getMem(1,1))


## An exact simplification method for Mem(X,Y)

In the case of a strategy of type `mem(X,Y)`, to know if this strategy can be scored with stars, we need to know if it uses all the elements of its code. If it does not, we identify these elements and replace them with stars. A star indicates that this gene can be replaced by any value since it is not used.
The function `getGenericName` takes as argument a strategy genotype and returns this genotype possibly rewritten with * .
The general idea of the algorithm used is to build little by little a list of all the possible pasts (starting from the possible pasts that the bootstrap allows), and thus, to put stars for the pasts which do not appear in this list. This is the case for example of `Mem(1,2, "ccCCCCCCCCCC")` which can thus be rewritten as `(1,2, "ccCCCCCC****")`.

In [ ]:
def setGenericName(strat):
    X = strat.x
    Y = strat.y
    genotype = strat.genome
    me = list(genotype[ max(X,Y) - X :  max(X,Y) ].upper())
    opponent=""
    L = [x for x in itertools.product(['C', 'D'],repeat=Y)]
    L = [list(elem) for elem in L]
    
    L1 = [x for x in itertools.product(['C', 'D'],repeat=X+Y)]
    L1 = [list(elem) for elem in L1]
  
    possiblePast = []
    tmp = []
    for elem in L :
        possiblePast += [me+elem] 
        tmp += [me+elem] 
    while(len(tmp)> 0):
        if Y > 0 : 
            hisPast = tmp[0][X + 1 : X + Y ]
        else : 
            hisPast = []
        myPast = []
        if X > 1: 
            myPast += tmp[0][1 : X]
        if X > 0 : 
            myPast += list(genotype[L1.index(tmp[0])+max(X,Y)].upper())
        if Y > 0 :
            past1 = myPast + hisPast + ['C']
            past2 = myPast + hisPast + ['D']
            #print(myPast)
            #print(hisPast)
        if Y == 0:
            past1 = myPast
            past2 = myPast 
            
        if past1 not in possiblePast :
            possiblePast += [past1]
            tmp += [past1]
        if past2 not in possiblePast :
            possiblePast += [past2]
            tmp += [past2]
        tmp.remove(tmp[0])
   
    # Recreate the genome
   
    ind=[]
    for i in range(len(L1)):
        if L1[i] not in possiblePast:
            ind += [i]
    
    genotypeStar = genotype[0 : max(X,Y)]
    for i in range(len(L1)):
        if i not in ind : 
            genotypeStar += genotype[max(X,Y) + i]
        else :
            genotypeStar += "*"
    return strat.clone(name = genotypeStar)



# example of use
print(setGenericName(Mem(1,2,"ccCCCCCCCC")).name)
# ccCCCC****

print(setGenericName(Mem(1,2,"cdCCCCDDDD")).name)
# ccCCDDCCDD

print(setGenericName(Mem(1,2,"ccCCCDCCDC")).name)
# ccCCCDCCDC

print(setGenericName(Mem(0,1,"cDD")).name)

print(setGenericName(Mem(1,0,"cCC")).name)

This result is important: it shows that, although the comparison of 2 programs is undecidable in the general case, it is nevertheless decidable in the case of `Mem(X,Y)`.

To test if two strategies of type `Mem(X,Y)` are identical, we just have to apply `getGenericName` and check that the names obtained are identical:

In [ ]:
s1 = Mem(2,2,'ccCCDDDCCDCDDDDDDD')
s2 = Mem(2,2,'ccCCDDDDDCDDDDDDCD') 

if (setGenericName(s1).name==setGenericName(s2).name) :
    print ("These two strategies are identical")
else: print ("These two strategies are different")

### General verification
In order to check the proper functioning of our different algorithms, we have all the tools to compare the result obtained with the exact approach (`setGenericName`) and with the approximate approach (`simplifyWithTournament` or `classesEquiv`, followed by `computeStar`).

To do this, we take a class (`Mem(1,2)` for example), then calculate its equivalence classes. For each non-empty class, we check that `setGenericName` on the key corresponds to the set of strategies of this equivalence class (in number by counting stars, or in quality by applying `computeStar` to the whole class).

In [ ]:
# simpl = simplifyWithTournament(getMem(1,2), [Periodic("CDC")], 20)
simpl = simplifyWithTournament(getMem(1,2), [Periodic('CDCCDDC'), Periodic('DDCDCDD')], 20)
for key in simpl:
    if len(simpl[key]) > 0 : 
        #print(key.name)
        #for strat in simpl[key]:
            #print(strat.name)
        name1 = computeStar([key] + simpl[key])
        nbStars = name1.count("*")
        # print("Check numbers of stars")
        if nbStars**2 != len(simpl[key])+1 :
            print("Stars problem with "+ name1)
            exit()
        # print("Check class")
        name2 = setGenericName(key).name
        for strat in simpl[key]:
            if name2 != setGenericName(strat).name :
                print("Key "+ name2 + " problem with its set "+ simpl[key])
                exit()
        # print("Check equivalence")
        if name1 != name2 :
            print(name1 +" different of " + name2)
            exit()
print("All is perfect")

# If we test with [Periodic("CDC"),Periodic("DCD")]  the result will be different


### Exercise 2

Write `allC` and `allD` in the `Mem(1,2)` formalism. Compute their names with * . 
Explain why these stars are not in the same place?

In [ ]:
allC = Mem(1,2,"ccCCCCCCCC")
allD = Mem(1,2,"ddDDDDDDDD")
print(setGenericName(allC).name)
print(setGenericName(allD).name)

### Simplification of a Memory set by the exact method

Now it remains to use this function to simplify a set of strategies of the Memory type: we take all the possible names, we browse the list name by name, we replace each name by its generic name and, if it is already present in the future result, we do not add it. 

Note that no tournament or match is necessary to simplify a class with this method! It is therefore possible to tackle much larger sets.

In [ ]:
def simplifyExact(bag):
    res = set()
    simplified = []
    for strat in bag : 
        gName = setGenericName(strat).name
        if gName not in res:
            res.add(gName)
            simplified += [strat]
    return simplified

In [ ]:
simplified = simplifyExact(getMem(1,2))
print("Lenght of bag simplified : {}".format(len(simplified)))

### Synthesis of simplications obtained

In [ ]:
tab = pd.DataFrame(
        np.nan, ["Mem(0,1)","Mem(1,0)","Mem(1,1)", "Mem(0,2)", "Mem(2,0)","Mem(2,1)","Mem(1,2)","Mem(2,2)","Mem(3,0)"], ["size","simplif"]
    )
l = getMem(0,1)
tab.at["Mem(0,1)","size"] = len(l)
tab.at["Mem(0,1)","simplif"] = len(simplifyExact(l))
l = getMem(1,0)
tab.at["Mem(1,0)","size"] = len(l)
tab.at["Mem(1,0)","simplif"] = len(simplifyExact(l))
l = getMem(1,1)
tab.at["Mem(1,1)","size"] = len(l)
tab.at["Mem(1,1)","simplif"] = len(simplifyExact(l))
l = getMem(0,2)
tab.at["Mem(0,2)","size"] = len(l)
tab.at["Mem(0,2)","simplif"] = len(simplifyExact(l))
l = getMem(2,0)
tab.at["Mem(2,0)","size"] = len(l)
tab.at["Mem(2,0)","simplif"] = len(simplifyExact(l))
l = getMem(2,1)
tab.at["Mem(2,1)","size"] = len(l)
tab.at["Mem(2,1)","simplif"] = len(simplifyExact(l))
l = getMem(1,2)
tab.at["Mem(1,2)","size"] = len(l)
tab.at["Mem(1,2)","simplif"] = len(simplifyExact(l))
l = getMem(2,2)
tab.at["Mem(2,2)","size"] = len(l)
tab.at["Mem(2,2)","simplif"] = len(simplifyExact(l))
l = getMem(3,0)
tab.at["Mem(3,0)","size"] = len(l)
tab.at["Mem(3,0)","simplif"] = len(simplifyExact(l))
l = getMem(0,3)
tab.at["Mem(0,3)","size"] = len(l)
tab.at["Mem(0,3)","simplif"] = len(simplifyExact(l))
l = getMem(3,1)
tab.at["Mem(3,1)","size"] = len(l)
tab.at["Mem(3,1)","simplif"] = len(simplifyExact(l))
l = getMem(1,3)
tab.at["Mem(1,3)","size"] = len(l)
tab.at["Mem(1,3)","simplif"] = len(simplifyExact(l))


tab

## A new method to compare a complete class and its simplified class

The comparison between a class and its simplified class had already been shown previously, using approximate methods. We do exactly the same thing again, but this time using the exact method. Remember that this exact method only works for Memory.


In [ ]:
# Simplified class computation
mem11 = getMem(1,1)
simpl = simplifyExact(mem11)
print(len(simpl))

# Replacement of names by generic names
bag = []
for strat in simpl:
    bag += [setGenericName(strat)]


# Evolution of the initial class
e1 = Ecological(g, mem11)
e1.run()

evol=e1.historic
nbSurvivors = len(evol.iloc[-1][evol.iloc[-1]>0])
e1.drawPlot(None,nbSurvivors)

# Evolution of the simplified class with its generic names
e2 = Ecological(g, bag)
e2.run()

evol=e2.historic
nbSurvivors = len(evol.iloc[-1][evol.iloc[-1]>0])
e2.drawPlot(None,10)


# Bibliography

- A. M. Turing. *On Computable Numbers, with an Application to the Entscheidungsproblem*. Proceedings of the London Mathematical Society, vol. s2-42, no 1, 1er janvier 1937, p. 230–265
- H. G. Rice. *Classes of Recursively Enumerable Sets and Their Decision Problems*. Transactions of the American Mathematical Society, volume 74, numéro 2, mars 1953 (see [Wikipedia](https://fr.wikipedia.org/wiki/Théorème_de_Rice))
- Bruno Beaufils, Jean-Paul Delahaye, Philippe Mathieu. *Complete classes of strategies for the Classical Iterated Prisoner's Dilemma*. Evolutionary Programming VII, 1998, Volume 1447. ISBN : 978-3-540-64891-8
- Stefan Ciobaca, Dorel Lucanu, Vlad Rusu, Grigore Rosu. *A Language-Independent Proof System for Full Program Equivalence*. Formal Aspects of Computing, Springer Verlag, 2016, 28 (3), pp.469–497.
